#### Import required Libraries

In [1]:
import pandas as pd
import os
os.system('pip install wikipedia')
os.system('pip install geocoder')
os.system('pip install geopy')
os.system('pip install folium')
import wikipedia as wp
from geopy.geocoders import Nominatim
import folium

#### Call the wikipedia page function and pass the "list of postal codes of Canada" page, read the html and create the df

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.columns = ['Postalcode', 'Borough', 'Neighborhood']
df = df.iloc[1:]
#df.to_csv('C:\\Users\RAGHUGOPALKRISHNAN\\Desktop\\Course9.csv')
#print(df)

#### Use groupby to group on the Postalcode column and aggregate on the Neighborhood column. Drop rows which have 'not assigned' in the Borough column and assign Borough value if the Neighborhood has value as 'Not assigned'

In [3]:
foo = lambda a: " , ".join(a) 
df = df.groupby('Postalcode').agg({'Borough': 'first', 'Neighborhood': foo}).reset_index()
   
    
df = df.drop(df[(df.Borough == 'Not assigned')].index)
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        #print(row['Borough'])
        row['Neighborhood'] = row['Borough']

#### Let's look at the shape of the Dataframe

In [4]:
print('The shape of the final Dataframe ', df.shape)

The shape of the final Dataframe  (103, 3)


#### Due to unreliability of the geocoder package I am using the geospatial csv file to get the latitude and longtitude for the Postal Codes

In [5]:
geospatial = pd.read_csv('https://cocl.us/Geospatial_data')
#geospatial
geospatial.columns = ['Postalcode', 'Latitude', 'Longitude']
#geospatial
df_merge = pd.merge(df, geospatial, on='Postalcode')
df_merge

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


#### Let's look at the shape of the final Dataframe

In [6]:
print('The shape of the final Dataframe ', df_merge.shape)

The shape of the final Dataframe  (103, 5)


#### Display number of Neighborhoods by Borough

In [7]:
df_merge.groupby('Borough').count()

,Postalcode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


#### Get Latitude and Longitude of Downtown Toronto

In [8]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


#### Create a map of Downtown Toronto with neighborhoods superimposed on top

In [9]:
from IPython.display import display
Downtown_Toronto = df_merge[df_merge['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.head()
Latitude = 43.654027
Longitude = -79.3802003
map_Downtown_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)
for lat, lng, label in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)
    
# map_Downtown_Toronto.save('map_Downtown_Toronto.html')
# f = codecs.open("map_Downtown_Toronto.html", 'r', 'utf-8')
# f
map_Downtown_Toronto

#### Define Foursquare credentials and version

In [10]:
CLIENT_ID = 'BLLCQDJ141BQCBPOYYMFN5VUIFH5ZR4NDTDFRP52QHQM2CPK' # your Foursquare ID
CLIENT_SECRET = 'G20BXB1EQAM0ZSJF35AOMV12LDF11ODIOH0XRVIATC0OLUZN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

#### Create a function to explore neighborhoods

In [11]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Explore each neighborhood using the above function and create a new Df called Downtown_Toronto_venues

In [12]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )
Downtown_Toronto_venues



Rosedale
Cabbagetown , St. James Town
Church and Wellesley
Harbourfront , Regent Park
Ryerson , Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide , King , Richmond
Harbourfront East , Toronto Islands , Union Station
Design Exchange , Toronto Dominion Centre
Commerce Court , Victoria Hotel
Harbord , University of Toronto
Chinatown , Grange Park , Kensington Market
CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place , Underground city
Christie


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown , St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"Cabbagetown , St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
6,"Cabbagetown , St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
7,"Cabbagetown , St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
8,"Cabbagetown , St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,"Cabbagetown , St. James Town",43.667967,-79.367675,Cabbagetown Brew,43.666923,-79.369289,Café


#### Check the size of the resulting dataframe

In [13]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()


(485, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown , St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Venues returned for each Neighborhood

In [14]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide , King , Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara",15,15,15,15,15,15
"Cabbagetown , St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown , Grange Park , Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court , Victoria Hotel",30,30,30,30,30,30


#### Unique categories curated from all the returned venues

In [15]:
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 146 uniques categories.


#### Analyze each Neighborhood

In [16]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]

Downtown_Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Dataframe size

In [17]:
Downtown_Toronto_onehot.shape

(485, 146)

#### Group rows by Neighborhood by taking mean of frequency of each category

In [18]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide , King , Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
2,"CN Tower , Bathurst Quay , Island airport , Ha...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown , St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,"Chinatown , Grange Park , Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.00,0.000000,0.066667,0.000000,0.066667,0.033333,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333
8,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Design Exchange , Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000


####  New DF size

In [ ]:
Downtown_Toronto_grouped.shape

#### Print each neighorhood with top 5 most common venues

In [19]:
num_top_venues = 5

for hood in Downtown_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_Toronto_grouped[Downtown_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond----
              venue  freq
0        Steakhouse  0.10
1             Hotel  0.07
2  Asian Restaurant  0.07
3       Pizza Place  0.07
4              Café  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1  Seafood Restaurant  0.07
2                Café  0.07
3      Farmers Market  0.07
4         Coffee Shop  0.03


----CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown , St. James Town----
                venue  freq
0  Italian Restaurant  0.07
1         Coffee Shop  0.07
2          Restaurant  0.07
3                Café  0.07
4   Indian Restaurant  0.03


----Central Bay Street----
                   venue  freq
0            Coffee Shop  0.23
1     Italian Restaurant  0.07
2        Bu

#### Function to sort venues

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create DF to display top 10 venues for each Neighborhood

In [23]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_Toronto_grouped['Neighborhood']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Steakhouse,Pizza Place,Café,Hotel,Asian Restaurant,American Restaurant,Sushi Restaurant,Bar,Plaza,Speakeasy
1,Berczy Park,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Museum,Steakhouse,Belgian Restaurant,Liquor Store,Beer Bar,Basketball Stadium
2,"CN Tower , Bathurst Quay , Island airport , Ha...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Sculpture Garden
3,"Cabbagetown , St. James Town",Restaurant,Italian Restaurant,Café,Coffee Shop,Park,Market,Pub,Playground,Pet Store,Butcher
4,Central Bay Street,Coffee Shop,Spa,Bubble Tea Shop,Italian Restaurant,Art Museum,Sandwich Place,Bar,Seafood Restaurant,Chinese Restaurant,Japanese Restaurant


#### Cluster Neighborhoods

In [26]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 2, 2, 0, 4, 0, 2, 2], dtype=int32)

#### Add clustering Lables

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_Toronto_merged = Downtown_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Wings Joint,College Gym,Creperie,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop
1,M4X,Downtown Toronto,"Cabbagetown , St. James Town",43.667967,-79.367675,2,Restaurant,Italian Restaurant,Café,Coffee Shop,Park,Market,Pub,Playground,Pet Store,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Gay Bar,Japanese Restaurant,Wings Joint,Mexican Restaurant,Ramen Restaurant,Pub,Bookstore,Breakfast Spot,Pizza Place,Bubble Tea Shop
3,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Mexican Restaurant,Yoga Studio,Event Space,Performing Arts Venue
4,M5B,Downtown Toronto,"Ryerson , Garden District",43.657162,-79.378937,0,Café,Clothing Store,Gastropub,Plaza,Hotel,Sandwich Place,Japanese Restaurant,Ramen Restaurant,Beer Bar,Comic Shop


#### Visualize the clusters

In [32]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_Toronto_merged['Latitude'], Downtown_Toronto_merged['Longitude'], Downtown_Toronto_merged['Neighborhood'], Downtown_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters